# Microwakings Model Single File
Training a model to detect microwakings, on a single file.
Superseeded by MicrowakingsMulti1.ipynb

In [2]:
%reload_ext autoreload
%autoreload 2
import logging
import os

log = lambda msg: logging.info(msg)


# Load data

In [3]:
input_file = "C:\\dev\\play\\brainwave-data\\2024-07-16-23-14-52\\raw.fif"
input_file_without_ext = os.path.splitext(input_file)[0]


In [4]:
# import sys
# import convert
# 
# raw, input_file_without_ext, mne_filtered = convert.load_mne_file(log, input_file)

In [5]:
import mne

raw = mne.io.read_raw_fif(input_file, preload=True)
filtered = raw.copy()

# AASM recommendation
filtered.filter(0.3, 35)

filtered.notch_filter(freqs=[50,100])

# Bit confused about this, something to do with MNE storing in volts.  But YASA complains it doesn't look uV if I don't do this.
data = filtered.get_data(units=dict(eeg="uV")) / 1_000_000
filtered._data = data
mne_filtered = filtered

input_file_without_ext = os.path.splitext(input_file)[0]


Opening raw data file C:\dev\play\brainwave-data\2024-07-16-23-14-52\raw.fif...
Isotrak not found
    Range : 0 ... 7403887 =      0.000 ... 29615.548 secs
Ready.
Reading 0 ... 7403887  =      0.000 ... 29615.548 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2751 samples (11.004 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin

In [6]:
import json
import pandas as pd

json_file_path = input_file.replace('.fif', '.scorings.json')
with open(json_file_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data, 'marks', errors='ignore')
df

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\dev\\play\\brainwave-data\\2024-07-16-23-14-52\\raw.scorings.json'

# Prepare wakings

In [ ]:
microwakings = df.copy()
# Convert timestamp and scoredAt to datetime
microwakings['timestamp'] = pd.to_datetime(microwakings['timestamp'])
microwakings['scoredAt'] = pd.to_datetime(microwakings['scoredAt'])

# Convert type and channel to string
microwakings['type'] = microwakings['type'].astype(str)
microwakings['channel'] = microwakings['channel'].astype(str)
microwakings

In [ ]:
# Identify the most frequent channel
most_frequent_channel = microwakings['channel'].value_counts().idxmax()

# Filter the DataFrame to only include rows with the most frequent channel
microwakings = microwakings[microwakings['channel'] == most_frequent_channel]

# Display the filtered DataFrame
microwakings

In [ ]:
import pandas as pd

# Initialize an empty list to store matched microwakings
matched_microwakings = []

# Loop through the DataFrame to find matching MicrowakingStart and MicrowakingEnd
for i, start_row in microwakings[microwakings['type'] == 'MicrowakingStart'].iterrows():
    start_time = start_row['timestamp']
    for j, end_row in microwakings[microwakings['type'] == 'MicrowakingEnd'].iterrows():
        end_time = end_row['timestamp']
        if start_time <= end_time <= start_time + pd.Timedelta(minutes=2):
            matched_microwakings.append((start_time, end_time))
            break  # Assuming one-to-one matching

# Convert the matched microwakings to a DataFrame if needed
microwakings_df = pd.DataFrame(matched_microwakings, columns=['Start', 'End'])
microwakings_df['Duration'] = microwakings_df['End'] - microwakings_df['Start']

# Display the matched DataFrame
microwakings_df

In [ ]:
microwakings_df['Duration'].describe()

# Prepare model data

In [ ]:
resampled = mne_filtered.copy()
# 100 hz is very similar to 250 hz to naked eye.
resampled_rate = 10
resampled.resample(resampled_rate, npad="auto")
eeg_data = resampled.get_data(picks = most_frequent_channel, units=dict(eeg="uV"))
eeg_data.shape


In [ ]:
import numpy as np
import pandas as pd

# Step 1: Extract start and end times from mne_filtered
start_time = mne_filtered.info['meas_date']
num_samples = eeg_data.shape[1]
end_time = start_time + pd.Timedelta(seconds=num_samples / resampled_rate)

# Step 2: Create a DataFrame with timestamps
timestamps = pd.date_range(start=start_time, periods=num_samples, freq=pd.Timedelta(seconds=1/resampled_rate))
labels_df = pd.DataFrame({'timestamp': timestamps, 'Microwaking': 0})

# Step 3: Label the timestamps
for _, row in microwakings_df.iterrows():
    labels_df.loc[(labels_df['timestamp'] >= row['Start']) & (labels_df['timestamp'] <= row['End']), 'Microwaking'] = 1

# Step 4: Convert the DataFrame to a NumPy array
labels = labels_df['Microwaking'].to_numpy()

# labels is now a NumPy array with the same number of rows as eeg_data, containing 1 for microwaking events and 0 otherwise

In [ ]:
labels_df

In [ ]:
labels_df['Microwaking'].value_counts()

In [ ]:
X = eeg_data.reshape((1, eeg_data.shape[1], 1))
y = labels.reshape((1, labels.shape[0], 1))
timesteps, num_features = X.shape[1], 1
#timesteps, num_features

y2 = np.reshape(y, (y.shape[1], 1))
X2 = np.reshape(X, (X.shape[1], X.shape[2]))
y2 = y2.reshape((1, y2.shape[0], 1))

X_reshaped = X.reshape((X.shape[1], 1))
y_reshaped = y2.reshape((X.shape[1], 1))
X = X_reshaped
y = y_reshaped

X.shape, y.shape


In [ ]:
timesteps, num_features

# Train model

In [ ]:
import tensorflow as tf

# Verify if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

In [ ]:
import sys
print(sys.version)
print(sys.executable)

In [ ]:
from tensorflow.keras import mixed_precision

# Set the mixed precision policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
tf.get_logger().setLevel(logging.WARNING)
tf.debugging.set_log_device_placement(False)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [ ]:
X.shape, y.shape

In [ ]:
import time
import datetime
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def high_recall_loss(y_true, y_pred):
    """Custom loss function that heavily penalizes false negatives"""
    y_true = tf.cast(y_true, tf.float16)
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

    # Standard binary crossentropy
    bce = y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred)

    # Additional penalty for false negatives
    false_negative_penalty = 10.0 * y_true * K.log(y_pred)

    return -K.mean(bce + false_negative_penalty, axis=-1)

# Assuming X and y are your full dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = Sequential([
    Input(shape=(364479, 1)),
    Conv1D(32, kernel_size=120, activation='relu', padding='same', name="Conv1D_1"),
    Conv1D(64, kernel_size=60, activation='relu', padding='same', name="Conv1D_2"),
    Conv1D(64, kernel_size=30, activation='relu', padding='same', name="Conv1D_3"),
    Conv1D(32, kernel_size=15, activation='relu', padding='same', name="Conv1D_4"),
    TimeDistributed(Dense(1, activation='sigmoid'))
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=high_recall_loss,
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print("tensorboard in " + log_dir)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


class LogToStdout(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}: loss = {logs['loss']:.4f}, accuracy = {logs['accuracy']:.4f}, val_loss = {logs['val_loss']:.4f}, val_accuracy = {logs['val_accuracy']:.4f}")

# Instantiate the callback
log_to_stdout = LogToStdout()

start_time = time.time()

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, tensorboard, log_to_stdout]
)

end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Trained in: {elapsed_time:.6f} seconds")

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix
# print(classification_report(y_val, adjusted_predictions))
# print(confusion_matrix(y_val, adjusted_predictions))
# 
# # Plot ROC curve to help choose optimal threshold
# from sklearn.metrics import roc_curve
# import matplotlib.pyplot as plt
# 
# fpr, tpr, thresholds = roc_curve(y_val.flatten(), predictions.flatten())
# plt.figure()
# plt.plot(fpr, tpr)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.show()

In [ ]:
# y_val.shape, predictions.shape

In [ ]:
# Assuming predictions and y_val are your model outputs and validation labels
#predictions = model.predict(X_val)
predictions = model.predict(X_train)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


# Ensure y_val is 2D and binary
y_val_reshaped = y_val.reshape(y_val.shape[0], -1)
y_val_binary = (y_val_reshaped > 0.5).astype(int)

# Reshape predictions to match y_val
adjusted_predictions = np.nan_to_num(predictions, nan=-1)
predictions_reshaped = adjusted_predictions.reshape(adjusted_predictions.shape[0], -1)

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_val_binary.ravel(), predictions_reshaped.ravel())
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Function to compute metrics at different thresholds
def compute_metrics_at_threshold(y_true, y_pred, threshold):
    y_pred_binary = (y_pred > threshold).astype(int)
    report = classification_report(y_true, y_pred_binary, output_dict=True)
    return report['1']['precision'], report['1']['recall'], report['1']['f1-score']

# Compute metrics at different thresholds
thresholds_to_try = [0.1, 0.3, 0.5, 0.7, 0.9]
for threshold in thresholds_to_try:
    precision, recall, f1 = compute_metrics_at_threshold(y_val_binary, predictions_reshaped, threshold)
    print(f"Threshold: {threshold:.2f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

# Choose a threshold (this is an example, you should choose based on your requirements)
chosen_threshold = 0.3
adjusted_predictions = (predictions_reshaped > chosen_threshold).astype(int)

# Final evaluation
print("\nFinal Evaluation:")
cr = classification_report(y_val_binary, adjusted_predictions)
print(cr)
print("Confusion Matrix:")
cm = confusion_matrix(y_val_binary, adjusted_predictions)
print(cm)

In [ ]:
cm

In [ ]:
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']

# Convert confusion matrix to a DataFrame with row/column labels
cm_df = pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'],
                     columns=['Predicted Negative', 'Predicted Positive'])

cm_df

In [ ]:
x = np.arange(len(predictions_reshaped))

plt.figure(figsize=(12, 6))
plt.plot(x, predictions_reshaped)

# Customize the plot
plt.title('Line Plot of Predictions')
plt.xlabel('Sample Index')
plt.ylabel('Prediction Value')

# Set y-axis limits
plt.ylim(0, 1)

# Add a grid for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
# predictions.shape, X_val.shape

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Input, GRU, GlobalAveragePooling1D
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from tensorflow.keras.layers import Conv1D, MaxPooling1D
# 
# 
# 
# # Split the dataset into training and testing sets
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# 
# # Build the LSTM-based sequence labeling model
# # model = Sequential()
# # 
# # model.add(Input(shape=(timesteps, num_features)))
# # 
# # # Extract features with a CNN that we will feed to the LSTM
# # model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# # model.add(MaxPooling1D(pool_size=2))
# # 
# # # GRU layer with return_sequences=True since we need a prediction for each time step
# # model.add(GRU(units=64, return_sequences=True))
# # 
# # # Dropout layer to avoid overfitting
# # model.add(Dropout(0.2))
# # 
# # # TimeDistributed Dense layer to predict a label for each time step (binary classification)
# # model.add(TimeDistributed(Dense(1, activation='sigmoid')))
# # 
# # # Compile the model with binary crossentropy for binary classification
# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# 
# model = Sequential([
#     Input(shape=(X_reshaped.shape[0], X_reshaped.shape[1])),
#     Conv1D(32, kernel_size=60, activation='relu', padding='same'),
#     Conv1D(64, kernel_size=15, activation='relu', padding='same'),
#     Conv1D(64, kernel_size=5, activation='relu', padding='same'),
#     Conv1D(32, kernel_size=3, activation='relu', padding='same'),
#     TimeDistributed(Dense(1, activation='sigmoid'))
# ])
# 
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
# 
# 
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)
# 
# class LogToStdout(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         print(f"Epoch {epoch + 1}: loss = {logs['loss']:.4f}, accuracy = {logs['accuracy']:.4f}, val_loss = {logs['val_loss']:.4f}, val_accuracy = {logs['val_accuracy']:.4f}")
# 
# # Instantiate the callback
# log_to_stdout = LogToStdout()
# 
# # Train the model
# history = model.fit(X_reshaped, y_reshaped, epochs=10, batch_size=32, callbacks=[early_stopping, tensorboard, log_to_stdout], validation_split=0.2)


In [ ]:
# probabilities = model.predict(X)

In [ ]:
# probabilities

In [ ]:
model.save('./microwakings1.h5')


In [ ]:
# import numpy as np
# from sklearn.metrics import accuracy_score
# 
# # Flatten the arrays
# probabilities_flat = probabilities.flatten()
# y_reshaped_flat = y_reshaped.flatten()
# 
# # Convert probabilities to binary predictions
# predictions = (probabilities_flat > 0.5).astype(int)
# 
# # Calculate accuracy
# accuracy = accuracy_score(y_reshaped_flat, predictions)
# print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# import numpy as np
# 
# # Ensure probabilities is reshaped to match the number of samples
# probabilities_channel = probabilities.flatten().reshape(1, -1)
# 
# # Find the indexes of non-finite values
# non_finite_indexes = np.where(~np.isfinite(probabilities_channel))[1]
# 
# # Get the non-finite values
# non_finite_values = probabilities_channel[0, non_finite_indexes]
# 
# # Print the indexes and values of non-finite values
# for idx, value in zip(non_finite_indexes, non_finite_values):
#     print(f"Index: {idx}, Value: {value}")

In [ ]:
probabilities = model.predict(X)

In [ ]:
import numpy as np
import mne

# Step 1: Extract the existing data from the resampled object
existing_data = resampled.get_data()

# Step 2: Add a new channel with the probabilities data
# Ensure probabilities is reshaped to match the number of samples
probabilities_channel = probabilities.flatten().reshape(1, -1)

# Step 3: Identify non-finite values and replace them with -50
non_finite_indexes = np.where(~np.isfinite(probabilities_channel))[1]
probabilities_channel[0, non_finite_indexes] = -50

# Step 4: Multiply all other values by 50
probabilities_channel = np.where(probabilities_channel == -50, -50, probabilities_channel * 50)

# Step 5: Concatenate the new channel to the existing data
new_data = np.vstack([existing_data, probabilities_channel])

# Step 6: Create a new Info object for the new channel
new_info = mne.create_info(
    ch_names=resampled.ch_names + ['probabilities'],
    sfreq=resampled.info['sfreq'],
    ch_types=resampled.get_channel_types() + ['misc']
)

# Step 7: Create a new RawArray with the updated data and info
new_raw = mne.io.RawArray(new_data, new_info)

# Step 8: Save the modified data to an EDF file
mne.export.export_raw(input_file_without_ext + ".with_predictions.edf", new_raw, overwrite=True)

In [ ]:
np.max(probabilities_channel)

In [ ]:
# import numpy as np
# 
# # Flatten the predictions array
# predictions_flat = predictions.flatten()
# 
# # Find the indexes of non-zero values
# non_zero_indexes = np.where(predictions_flat != 0)[0]
# 
# # Get the non-zero values
# non_zero_values = predictions_flat[non_zero_indexes]
# 
# # Print the indexes and values of non-zero values
# for idx, value in zip(non_zero_indexes, non_zero_values):
#     print(f"Index: {idx}, Value: {value}")

In [ ]:
# import numpy as np
# 
# # Flatten the predictions array
# predictions_flat = predictions.flatten()
# 
# # Find the indexes of non-zero values
# non_zero_indexes = np.where(predictions_flat != 0)[0]
# 
# # Print the total count of non-zero values
# print(f"Total count of non-zero values in predictions: {len(non_zero_indexes)}")
# 
# # Flatten the y_reshaped array
# y_reshaped_flat = y_reshaped.flatten()
# 
# # Find the indexes of non-zero values
# non_zero_indexes_y = np.where(y_reshaped_flat != 0)[0]
# 
# # Print the total count of non-zero values
# print(f"Total count of non-zero values in y_reshaped: {len(non_zero_indexes_y)}")

In [ ]:
# import numpy as np
# 
# # Replace NaN values in y_reshaped with -1
# y_reshaped_2 = np.nan_to_num(y_reshaped, nan=-1)
# adjusted_predictions_2 = np.nan_to_num(adjusted_predictions, nan=-1)
# 
# print(classification_report(y_reshaped_2, adjusted_predictions_2))


In [ ]:
# print(confusion_matrix(y_reshaped_2, adjusted_predictions_2))


In [ ]:
# pd.DataFrame(adjusted_predictions_2.reshape(-1, 1)).value_counts()